# My Submission: Part One

This is part one of my week 3 submission. I will scrape the Wikipedia page and then clean the data based on the submission requirements:

<b> To create the dataframe:</b>
<ul>
    <li> The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    <li> Only process the cells that have an assigned borough. Ignore cells with a borough that is <b>Not assigned</b>.
    <li> More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that <b>M5A</b> is listed twice and has two neighborhoods: <b>Harbourfront</b> and <b>Regent Park</b>. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in <b>row 11</b> in the above table.
    <li> If a cell has a borough but a <b>Not assigned</b> neighborhood, then the neighborhood will be the same as the borough. So for the <b>9th</b> cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be <b>Queen's Park</b>.
    <li> Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    <li> In the last cell of your notebook, use the <b>.shape</b> method to print the number of rows of your dataframe.
</ul>

### Importing necessary libraries
<ul>
    <li> <b>Numpy</b>
    <li> <b>Pandas</b>
    <li> <b>Requests</b>
    <li> <b>lxml</b>
</ul>

In [1]:
# Importing all necessary libraries from the "Segmenting and Clustering Neighborhoods in New York City" lab
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Importing the Web Scraping libraries
# Install these into your system/environment if not downloaded
import requests
import lxml.html as lh

print('Libraries imported.')

Libraries imported.


In [2]:
# The url for the wiki page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

### Scraping the Wikipedia page using code found online 

<b>Link:</b> https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
I changed one part of the code to better fit our purposes

  <i>"If row is not of size 10, the //tr data is not from our table 
    
    if len(T)!=10:"

   </i>I changed the <b>10</b> to <b>3</b> since we have 3 columns, and their example used 10 columns

In [3]:
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

Here I cleaned up the data based on the requirements:
- Removing the '\n' from the last column since this character was in every data cell.
- Changed the Column Names and removing the 'Not assigned' rows
- Set Index and Grouped the dataframe by PostalCode AND Borough then Joining the Neighborhood by a comma. I changed the index to better group the columns and rows
- Resetting the index back to integers

In [4]:
# Removing the '\n' from the last column since this character was in every data cell.
df[col[2][0]] = df[col[2][0]].astype(str).str[:-1]

# Changed the Column Names and removing the 'Not assigned' rows
df.columns = ['PostalCode', 'Borough','Neighborhood']
df = df[df.Borough != 'Not assigned']

# Grouping the dataframe by PostalCode AND Borough then Joining the Neighborhood by a comma
df.set_index(['PostalCode', 'Borough'],inplace=True)
df = df.groupby(level=['PostalCode','Borough'], sort=False).agg( ', '.join)

# Resetting the index back to integers
df.reset_index(inplace=True)

In [5]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


### Shape of the dataframe

In [6]:
df.shape

(103, 3)